<a href="https://colab.research.google.com/github/sainath241/Optimization-of-Energy-Consumption-in-Smart-Home-using-Machine-Learning-/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyngrok # Install the pyngrok module
!pip install flask-caching # Install the flask-caching module

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from flask import Flask, render_template_string, redirect, url_for
from pyngrok import ngrok, conf
from flask_caching import Cache
from sklearn.covariance import EllipticEnvelope
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scipy.optimize import linprog
from google.colab import drive

# Mount Google Drive to access your data file and image
drive.mount('/content/drive')

# Initialize Flask application and configure caching
app = Flask(__name__, static_folder='/content/drive/MyDrive/Data/plots')
cache = Cache(app, config={'CACHE_TYPE': 'SimpleCache', 'CACHE_DEFAULT_TIMEOUT': 300})

smart_home = None
plot_size = (5, 3)
corr_plot_size = (15, 12)
scatter_plot_size = (15, 10)

def preprocess_data():
    global smart_home
    file_path = '/content/drive/MyDrive/Data/HomeC.csv'
    smart_home = pd.read_csv(file_path)

    smart_home['time'] = pd.DatetimeIndex(pd.date_range('2016-01-01 05:00', periods=len(smart_home), freq='min'))
    smart_home['year'] = smart_home['time'].apply(lambda x: x.year)
    smart_home['month'] = smart_home['time'].apply(lambda x: x.month)
    smart_home['day'] = smart_home['time'].apply(lambda x: x.day)
    smart_home['weekday'] = smart_home['time'].apply(lambda x: x.day_name())
    smart_home['weekofyear'] = smart_home['time'].apply(lambda x: x.isocalendar()[1])
    smart_home['hour'] = smart_home['time'].apply(lambda x: x.hour)
    smart_home['minute'] = smart_home['time'].apply(lambda x: x.minute)

    def hours2timing(x):
        if x in [20, 21, 22, 23, 0, 1, 2, 3, 4]:
            return 'Night'
        elif x in range(4, 12):
            return 'Morning'
        elif x in range(12, 16):
            return 'Afternoon'
        elif x in range(16, 20):
            return 'Evening'
        else:
            return 'X'

    smart_home['timing'] = smart_home['hour'].apply(hours2timing)
    smart_home.columns = [i.replace(' [kW]', '') for i in smart_home.columns]
    smart_home['Furnace'] = smart_home[['Furnace 1', 'Furnace 2']].sum(axis=1)
    smart_home['Kitchen'] = smart_home[['Kitchen 12', 'Kitchen 14', 'Kitchen 38']].sum(axis=1)
    smart_home.drop(['Furnace 1', 'Furnace 2', 'Kitchen 12', 'Kitchen 14', 'Kitchen 38', 'icon', 'summary'], axis=1, inplace=True)
    smart_home = smart_home[0:-1]

    numeric_cols = smart_home.select_dtypes(include=["float64", "int64"])
    outlier_detector = EllipticEnvelope(contamination=.1)
    outlier_detector.fit(numeric_cols)
    outlier_predictions = outlier_detector.predict(numeric_cols)
    smart_home["outlier"] = outlier_predictions

    smart_home['overall_energy_consumption'] = smart_home['use']
    smart_home['overall_energy_generation'] = smart_home['gen']
    output_csv_path = '/content/drive/MyDrive/Data/smart_home_data_preprocessed.csv'
    smart_home.to_csv(output_csv_path, index=False)

    generate_plots(smart_home)

def generate_plots(data):
    plot_dir = '/content/drive/MyDrive/Data/plots'
    if not os.path.exists(plot_dir):
        os.makedirs(plot_dir)
    #file path
    file_path = '/content/drive/MyDrive/Data/smart_home_data_preprocessed.csv'
    data = pd.read_csv(file_path)
    if 'Unnamed: 0' in data.columns:
        data = data.drop(columns=['Unnamed: 0'])

    # Boxplot
    numeric_cols = data.select_dtypes(include=["float64", "int64"])
    plt.figure(figsize=(6,4))
    numeric_cols.boxplot(rot=90)
    plt.title('Boxplot')
    plt.subplots_adjust(bottom=0.4)
    plt.savefig(f'{plot_dir}/boxplot.png')
    plt.close()

    # Line plot: Average Energy Consumption by Month
    grouped_df = data.groupby('month')['use'].mean().reset_index()
    plt.figure(figsize=plot_size)
    plt.plot(grouped_df['month'], grouped_df['use'], marker='o')
    plt.title('Average Energy Consumption by Month')
    plt.xlabel('Month')
    plt.ylabel('Energy Consumption in KW')
    plt.xticks(grouped_df['month'])
    plt.grid(True)
    plt.savefig(f'{plot_dir}/avg_energy_consumption_month.png')
    plt.close()

    # Line plot: Average Weekly Energy Consumption
    grouped_df_week = data.groupby('weekofyear')['use'].mean().reset_index()
    plt.figure(figsize=plot_size)
    plt.plot(grouped_df_week['weekofyear'], grouped_df_week['use'], marker='o')
    plt.title('Average Weekly Energy Consumption')
    plt.xlabel('Week')
    plt.ylabel('Energy Consumption in KW')
    plt.xticks(grouped_df_week['weekofyear'])
    plt.grid(True)
    plt.savefig(f'{plot_dir}/avg_weekly_energy_consumption.png')
    plt.close()

    # Line plot: Average Energy Generation by Month
    grouped_df_gen = data.groupby('month')['gen'].mean().reset_index()
    plt.figure(figsize=plot_size)
    plt.plot(grouped_df_gen['month'], grouped_df_gen['gen'], marker='o')
    plt.title('Average Energy Generation by Month')
    plt.xlabel('Month')
    plt.ylabel('Energy Generation in KW')
    plt.xticks(grouped_df_gen['month'])
    plt.grid(True)
    plt.savefig(f'{plot_dir}/avg_energy_generation_month.png')
    plt.close()

    # Line plot: Average Weekly Energy Generation
    grouped_df_week_gen = data.groupby('weekofyear')['gen'].mean().reset_index()
    plt.figure(figsize=plot_size)
    plt.plot(grouped_df_week_gen['weekofyear'], grouped_df_week_gen['gen'], marker='o')
    plt.title('Average Weekly Energy Generation')
    plt.xlabel('Week')
    plt.ylabel('Energy Generation in KW')
    plt.xticks(grouped_df_week_gen['weekofyear'])
    plt.grid(True)
    plt.savefig(f'{plot_dir}/avg_weekly_energy_generation.png')
    plt.close()

    # Bar chart: Energy Consumption by Appliance
    columns_of_interest = ['Dishwasher', 'Home office', 'Fridge', 'Wine cellar', 'Garage door', 'Barn', 'Well', 'Microwave', 'Living room', 'Furnace', 'Kitchen']
    energy_consumption = data[columns_of_interest].sum()
    plt.figure(figsize=plot_size)
    energy_consumption.plot(kind='bar', color='skyblue')
    plt.title('Energy Consumption by Appliance in kW')
    plt.xlabel('Appliance')
    plt.ylabel('Total Energy Consumption')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{plot_dir}/energy_consumption_appliance.png')
    plt.close()

    # Line plot: Daily Energy Usage
    data = data.reset_index()
    data['time'] = pd.to_datetime(data['time'])
    data.set_index('time', inplace=True)
    daily_data = data['use'].resample('D').sum()
    plt.figure(figsize=plot_size)
    daily_data.plot()
    plt.title('Daily Energy Usage')
    plt.xlabel('Date')
    plt.ylabel('Energy Usage')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{plot_dir}/daily_energy_usage.png')
    plt.close()

    # Line plot: Daily Energy Generation
    daily_data = data['gen'].resample('D').sum()
    plt.figure(figsize=plot_size)
    daily_data.plot()
    plt.title('Daily Energy Generation')
    plt.xlabel('Date')
    plt.ylabel('Energy Generation in KW')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{plot_dir}/daily_energy_generation.png')
    plt.close()

    # Line plots: Daily Temperature and Energy Use
    daily_data = data.resample('D').agg({'temperature': 'mean', 'use': 'sum'})
    plt.figure(figsize=(8,6))
    plt.subplot(2, 1, 1)
    plt.plot(daily_data.index, daily_data['temperature'], label='Temperature', color='orange')
    plt.title('Daily Temperature and Energy Use')
    plt.ylabel('Temperature (°C)')
    plt.legend()
    plt.subplot(2, 1, 2)
    plt.plot(daily_data.index, daily_data['use'], label='Energy Use', color='blue')
    plt.xlabel('Date')
    plt.ylabel('Energy Use (kWh)')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'{plot_dir}/temperature_energy_use.png')
    plt.close()

    # Scatter plots: Use vs. weather features
    columns_of_interest = ['temperature', 'humidity', 'visibility', 'apparentTemperature', 'pressure', 'windSpeed', 'windBearing', 'precipIntensity', 'dewPoint', 'precipProbability']
    plt.figure(figsize=scatter_plot_size)
    for i, column in enumerate(columns_of_interest, 1):
        plt.subplot(2, 5, i)
        plt.scatter(data[column], data['use'], alpha=0.5)
        plt.title(f'use vs {column}')
        plt.xlabel(column)
        plt.ylabel('use')
    plt.tight_layout()
    plt.savefig(f'{plot_dir}/scatter_plots.png')
    plt.close()

    # Correlation heatmap
    data_corr = data.drop(columns=['year', 'overall_energy_consumption', 'overall_energy_generation', 'outlier', 'Solar', 'House overall', 'month', 'day', 'weekday', 'weekofyear', 'hour', 'minute'])
    numerical_data = data_corr.select_dtypes(include=['float64', 'int64'])
    correlation_matrix = numerical_data.corr()
    plt.figure(figsize=corr_plot_size)
    sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
    plt.title('Correlation Matrix')
    plt.savefig(f'{plot_dir}/correlation_matrix.png')
    plt.close()

    smart_home_data_preprocessed_path = '/content/drive/MyDrive/Data/smart_home_eda.csv'
    data.to_csv(smart_home_data_preprocessed_path, index=True)

def predict_and_optimize():
    file_path = '/content/drive/MyDrive/Data/smart_home_eda.csv'
    smart_home_lstm = pd.read_csv(file_path)
    #  Drop the 'index' column
    if 'index' in smart_home_lstm.columns:
        smart_home_lstm = smart_home_lstm.drop(columns=['index'])

    if 'time' not in smart_home_lstm.columns:
       raise ValueError("'time' column is missing from the data.")

    smart_home_lstm['time'] = pd.to_datetime(smart_home_lstm['time'])
    smart_home_lstm.set_index('time', inplace=True)

    numeric_columns = smart_home_lstm.select_dtypes(include=np.number).columns
    smart_home_lstm = smart_home_lstm[numeric_columns].resample('H').mean()

    smart_home_lstm['weekday'] = LabelEncoder().fit_transform(pd.Series(smart_home_lstm.index).apply(lambda x: x.day_name())).astype(np.int8)
    smart_home_lstm['timing'] = LabelEncoder().fit_transform(smart_home_lstm['hour'].apply(lambda x: 'Morning' if 5 <= x < 12 else 'Afternoon' if 12 <= x < 18 else 'Evening' if 18 <= x < 22 else 'Night')).astype(np.int8)

    features = ['use', 'temperature', 'humidity', 'windSpeed', 'pressure']
    target = 'use'

    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(smart_home_lstm[features])

    def create_sequences(smart_home_lstm, input_steps=24, output_steps=1):
        X, y = [], []
        for i in range(len(smart_home_lstm) - input_steps - output_steps + 1):
            X.append(smart_home_lstm[i:i+input_steps])
            y.append(smart_home_lstm[i+input_steps:i+input_steps+output_steps, 0])
        return np.array(X), np.array(y)

    input_steps = 24
    output_steps = 1
    X, y = create_sequences(scaled_data, input_steps, output_steps)

    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(50))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse')
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)
    loss = model.evaluate(X_test, y_test)
    predictions = model.predict(X_test)

    y_test_scaled = scaler.inverse_transform(np.concatenate((y_test, np.zeros((y_test.shape[0], scaled_data.shape[1]-1))), axis=1))[:,0]
    predictions_scaled = scaler.inverse_transform(np.concatenate((predictions, np.zeros((predictions.shape[0], scaled_data.shape[1]-1))), axis=1))[:,0]

    last_24_hours = scaled_data[-24:].reshape(1, 24, len(features))
    next_hour_prediction = model.predict(last_24_hours)
    next_hour_prediction_scaled = scaler.inverse_transform(np.concatenate((next_hour_prediction, np.zeros((next_hour_prediction.shape[0], scaled_data.shape[1]-1))), axis=1))[:,0]

    threshold = 4.0
    optimized_values_str = ""
    optimal_use_value = 0
    if next_hour_prediction_scaled[0] > threshold:
        last_row = smart_home_lstm.iloc[-1]
        appliances_06_09 = ['Furnace', 'Barn', 'Fridge']
        appliances_02_04 = ['Dishwasher', 'Home office', 'Wine cellar', 'Garage door', 'Well', 'Microwave', 'Living room', 'Kitchen']
        c = np.ones(len(appliances_06_09) + len(appliances_02_04))
        bounds = [(0.6, 0.9) for _ in appliances_06_09] + [(0.2, 0.4) for _ in appliances_02_04]
        x0 = np.array([last_row[appliance] for appliance in appliances_06_09 + appliances_02_04])
        result = linprog(c, bounds=bounds, method='highs')
        optimized_values = result.x
        optimized_total = np.sum(optimized_values)
        optimized_values_str = "Optimized appliance values:<br>"
        for appliance, value in zip(appliances_06_09 + appliances_02_04, optimized_values):
            optimized_values_str += f"{appliance}: {value:.2f} kW<br>"
            if value == 0:
                optimized_values_str += f"Please turn off {appliance} to achieve the optimal consumption.<br>"
        optimal_use_value = optimized_total
        optimized_values_str += f"Optimized total consumption: {optimized_total:.2f} kW<br>"
        if optimized_total > threshold:
            optimized_values_str += "Even after optimization, the consumption is above the threshold. Consider reducing the use of major appliances or adjusting thermostat settings.<br>"
        else:
            optimized_values_str += "Optimization successful. The new consumption is within the threshold.<br>"
    else:
        optimized_values_str += "Total predicted consumption of next hour is below the threshold value so no need to turn off any appliances.<br>"

    plt.figure(figsize=(14, 5))
    plt.plot(smart_home_lstm.index[-len(y_test_scaled):], y_test_scaled, color='blue', label='Actual Consumption')
    plt.plot(smart_home_lstm.index[-len(y_test_scaled):], predictions_scaled, color='red', label='Predicted Consumption')
    plt.title('Electricity Consumption Prediction')
    plt.xlabel('Time')
    plt.ylabel('Consumption')
    plt.legend()
    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/Data/plots/predictions_vs_actual.png')
    plt.close()

    return optimized_values_str, next_hour_prediction_scaled[0], optimal_use_value

@app.route('/')
@cache.cached()
def index():
    return render_template_string('''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Optimization of Electricity</title>
            <style>
                body {
                    font-family: Arial, sans-serif;
                    margin: 0;
                    padding: 0;
                    background-color: white;
                    color: black;
                }
                .header {
                    background-image: url("{{ url_for('static', filename='smart_home_bg.jpg') }}");
                    background-size: cover;
                    color: white;
                    text-align: center;
                    padding: 50px 0;
                }
                .header h1 {
                    margin: 0;
                }
                .content {
                    text-align: center;
                    padding: 20px;
                    background-color: white;
                    color: black;
                }
                button {
                    background-color: #4CAF50;
                    color: white;
                    padding: 15px 32px;
                    text-align: center;
                    text-decoration: none;
                    display: inline-block;
                    font-size: 16px;
                    margin: 4px 2px;
                    cursor: pointer;
                    border: none;
                    border-radius: 5px;
                }
                table {
                    width: 100%;
                    border-collapse: collapse;
                    margin: 20px 0;
                }
                th, td {
                    padding: 15px;
                    text-align: left;
                    border-bottom: 1px solid #ddd;
                }
                th {
                    background-color: #4CAF50;
                    color: white;
                }
            </style>
        </head>
        <body>
            <div class="header">
                <h1>Optimization of Electricity</h1>
                <form action="/get" method="post">
                    <button type="submit">Get Data</button>
                </form>
            </div>
            <div class="content">
                <h2>Welcome to the Optimization of Electricity</h2>
                <p>Click the button above to get and preprocess the data.</p>
            </div>
        </body>
        </html>
    ''')

@app.route('/get', methods=['POST'])
def get():
    preprocess_data()
    return redirect(url_for('show_data'))

@app.route('/data')
@cache.cached()
def show_data():
    global smart_home
    data_html = smart_home.head(3).to_html(classes='data', header="true")
    optimized_values_str, next_hour_prediction, optimal_use_value = predict_and_optimize()
    return render_template_string('''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Smart Home Data</title>
            <style>
                body {
                    font-family: Arial, sans-serif;
                    margin: 0;
                    padding: 0;
                    background-color: white;
                    color: black;
                }
                .header {
                    background-image: url("{{ url_for('static', filename='background.jpg') }}");
                    background-size: cover;
                    color: white;
                    text-align: center;
                    padding: 50px 0;
                }
                .header h1 {
                    margin: 0;
                }
                .content {
                    text-align: center;
                    padding: 20px;
                    background-color: white;
                    color: black;
                }
                button {
                    background-color: #4CAF50;
                    color: white;
                    padding: 15px 32px;
                    text-align: center;
                    text-decoration: none;
                    display: inline-block;
                    font-size: 16px;
                    margin: 4px 2px;
                    cursor: pointer;
                    border: none;
                    border-radius: 5px;
                }
                table {
                    width: 100%;
                    border-collapse: collapse;
                    margin: 20px 0;
                }
                th, td {
                    padding: 15px;
                    text-align: left;
                    border-bottom: 1px solid #ddd;
                }
                th {
                    background-color: #4CAF50;
                    color: white;
                }
            </style>
        </head>
        <body>
            <div class="header">
                <h1>Smart Home Data</h1>
                <form action="/get" method="post">
                    <button type="submit">Get Data</button>
                </form>
            </div>
            <div class="content">
                <h2>First 3 Rows of Data</h2>
                <div>{{ data|safe }}</div>
                <h2>EDA Plots</h2>
                <img src="{{ url_for('static', filename='boxplot.png') }}" alt="Boxplot">
                <img src="{{ url_for('static', filename='avg_energy_consumption_month.png') }}" alt="Average Energy Consumption by Month">
                <img src="{{ url_for('static', filename='avg_weekly_energy_consumption.png') }}" alt="Average Weekly Energy Consumption">
                <img src="{{ url_for('static', filename='avg_energy_generation_month.png') }}" alt="Average Energy Generation by Month">
                <img src="{{ url_for('static', filename='avg_weekly_energy_generation.png') }}" alt="Average Weekly Energy Generation">
                <img src="{{ url_for('static', filename='energy_consumption_appliance.png') }}" alt="Energy Consumption by Appliance">
                <img src="{{ url_for('static', filename='daily_energy_usage.png') }}" alt="Daily Energy Usage">
                <img src="{{ url_for('static', filename='daily_energy_generation.png') }}" alt="Daily Energy Generation">
                <img src="{{ url_for('static', filename='temperature_energy_use.png') }}" alt="Daily Temperature and Energy Use">
                <img src="{{ url_for('static', filename='scatter_plots.png') }}" alt="Scatter Plots">
                <img src="{{ url_for('static', filename='correlation_matrix.png') }}" alt="Correlation Matrix">
                <h2>Predictions and Optimization</h2>
                <img src="{{ url_for('static', filename='predictions_vs_actual.png') }}" alt="Predictions vs Actual">
                <p>Next hour predicted consumption: {{ next_hour_prediction }} kW</p>
                <div>{{ optimized_values_str | safe }}</div>
                <p>Optimal use value: {{ optimal_use_value }} kW</p>
            </div>
        </body>
        </html>
    ''', data=data_html, next_hour_prediction=next_hour_prediction, optimized_values_str=optimized_values_str, optimal_use_value=optimal_use_value)

if __name__ == '__main__':
    # Configure Ngrok authtoken and start the Flask app
    conf.get_default().region = "us"

    # Start ngrok in a separate thread to avoid blocking the main thread
    import threading
    def start_ngrok():
        public_url = ngrok.connect(5000)
        print("Ngrok tunnel URL:", public_url)
    threading.Thread(target=start_ngrok).start()

    # Give ngrok some time to start before running the Flask app
    import time
    time.sleep(5)

    app.run()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ngrok tunnel URL: NgrokTunnel: "https://9c49-34-127-87-251.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/Jun/2024 04:45:06] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jun/2024 04:45:06] "GET /plots/smart_home_bg.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jun/2024 04:45:06] "GET /favicon.ico HTTP/1.1" 404 -
<ipython-input-9-aa9f5baa1ef4>:31: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  smart_home = pd.read_csv(file_path)
<ipython-input-9-aa9f5baa1ef4>:80: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)
INFO:werkzeug:127.0.0.1 - - [23/Jun/2024 04:47:19] "POST /get HTTP/1.1" 302 -
<ipython-input-9-aa9f5baa1ef4>:226: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set l

Epoch 1/20
65/65 [==============================] - 8s 44ms/step - loss: 0.0165 - val_loss: 0.0107
Epoch 2/20
65/65 [==============================] - 2s 28ms/step - loss: 0.0129 - val_loss: 0.0090
Epoch 3/20
65/65 [==============================] - 2s 28ms/step - loss: 0.0125 - val_loss: 0.0088
Epoch 4/20
65/65 [==============================] - 2s 38ms/step - loss: 0.0120 - val_loss: 0.0096
Epoch 5/20
65/65 [==============================] - 3s 45ms/step - loss: 0.0116 - val_loss: 0.0082
Epoch 6/20
65/65 [==============================] - 3s 45ms/step - loss: 0.0106 - val_loss: 0.0074
Epoch 7/20
65/65 [==============================] - 2s 29ms/step - loss: 0.0087 - val_loss: 0.0066
Epoch 8/20
65/65 [==============================] - 2s 28ms/step - loss: 0.0079 - val_loss: 0.0066
Epoch 9/20
65/65 [==============================] - 2s 29ms/step - loss: 0.0073 - val_loss: 0.0069
Epoch 10/20
65/65 [==============================] - 3s 40ms/step - loss: 0.0077 - val_loss: 0.0061
Epoch 11/

INFO:werkzeug:127.0.0.1 - - [23/Jun/2024 04:48:52] "GET /data HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jun/2024 04:48:53] "GET /plots/boxplot.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jun/2024 04:48:53] "GET /plots/background.jpg HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Jun/2024 04:48:53] "GET /plots/avg_energy_consumption_month.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jun/2024 04:48:53] "GET /plots/energy_consumption_appliance.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jun/2024 04:48:53] "GET /plots/avg_energy_generation_month.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jun/2024 04:48:53] "GET /plots/avg_weekly_energy_generation.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jun/2024 04:48:53] "GET /plots/avg_weekly_energy_consumption.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jun/2024 04:48:53] "GET /plots/daily_energy_generation.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jun/2024 04:48:53] "GET /plots/daily_e